# Perfilamiento con time

In [12]:
import time

In [16]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [15]:
#!pip uninstall mex -y

Found existing installation: mex 0.2
Uninstalling mex-0.2:
  Successfully uninstalled mex-0.2


In [90]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## Objetivo

In [18]:
c_max_obj_1 = [-3, -5]
A_max_obj_1 = [[1, 0], [0, 2], [3, 2]]
b_max_obj_1 = [4, 12, 18]

In [19]:
max_obj_1 = -1*linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).fun

In [20]:
coeff_obj_1 = linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).x

## Aproximación

In [50]:
n_var_approx_1 = 2
n_cons_approx_1 = 3

In [16]:
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
end_time = time.time()
secs = end_time-start_time
print("create_matrix tomó",secs,"segundos" )

create_matrix tomó 7.510185241699219e-05 segundos


In [17]:
start_time = time.time()
constrain(matrix_max_approx_1,'1,0,L,4')
end_time = time.time()
secs = end_time-start_time
print("constrain tomó",secs,"segundos" )

constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')

constrain tomó 0.0002276897430419922 segundos


In [18]:
start_time = time.time()
obj(matrix_max_approx_1,'3,5,0')
end_time = time.time()
secs = end_time-start_time
print("obj tomó",secs,"segundos" )

obj tomó 0.00018906593322753906 segundos


In [19]:
start_time = time.time()
problem_approx_1 = maxz(matrix_max_approx_1)
end_time = time.time()
secs = end_time-start_time
print("maxz tomó",secs,"segundos" )

maxz tomó 0.0005278587341308594 segundos


In [20]:
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))

In [21]:
# Todo junto
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
constrain(matrix_max_approx_1,'1,0,L,4')
constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')
obj(matrix_max_approx_1,'3,5,0')
problem_approx_1 = maxz(matrix_max_approx_1)
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))
end_time = time.time()
secs = end_time-start_time
print("Todo el proceso tomó",secs,"segundos" )

Todo el proceso tomó 0.0012271404266357422 segundos


## Comprobación

In [22]:
assert max_obj_1 == approx(max_approx_1), "El valor aproximado es incorrecto"
assert coeff_obj_1 == approx(coeff_approx_1), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_1)
print("El valor aproximado obtenido con mex es: ", max_approx_1)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_1)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1)

El valor objetivo obtenido con scipy es:  35.99999997843401
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


# Con comandos magic

Sólo probaremos con maxz que es la función que más se tardó en el proceso anterior

In [23]:
%time maxz(matrix_max_approx_1)

CPU times: user 315 µs, sys: 128 µs, total: 443 µs
Wall time: 447 µs


{'x1': 2.0, 'x2': 6.0, 'max': 36.0}

In [24]:
%timeit -n 5 -r 10 maxz(matrix_max_approx_1)

342 µs ± 14.4 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


# Con cProfile

In [25]:
import cProfile

In [26]:
cprof = cProfile.Profile()
cprof.enable()
problem_approx_1 = maxz(matrix_max_approx_1)
cprof.disable()
cprof.print_stats(sort='cumtime')

         212 function calls (192 primitive calls) in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.001    0.000 interactiveshell.py:3400(run_code)
        2    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-26-4199b605bc38>:3(<module>)
     21/1    0.000    0.000    0.001    0.001 line_profiler.py:124(wrapper)
        1    0.000    0.000    0.000    0.000 problem_definition.py:184(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:154(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:183(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:98(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:51(pivots_row)
       16    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        2    0.0

## con comando magic

In [27]:
 %prun -s cumulative problem_approx_1 = maxz(matrix_max_approx_1)

         170 function calls (150 primitive calls) in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
     21/1    0.000    0.000    0.001    0.001 line_profiler.py:124(wrapper)
        1    0.000    0.000    0.001    0.001 problem_definition.py:184(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:154(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:183(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:98(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:51(pivots_row)
       16    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        1    0.000    0.000    0.000    0.000 simplex_networks.py:29(pivots_col)
        6    0.000    0.000    0.000   

In [28]:
cprof.dump_stats("maxz_stats")

In [29]:
import pstats

In [30]:
p_max_stats = pstats.Stats("maxz_stats")
print(p_max_stats.sort_stats("cumulative").print_stats(10))

Mon May 10 01:38:18 2021    maxz_stats

         212 function calls (192 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 41 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.001    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400(run_code)
        2    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-26-4199b605bc38>:3(<module>)
     21/1    0.000    0.000    0.001    0.001 /home/ubuntu/.local/lib/python3.8/site-packages/line_profiler/line_profiler.py:124(wrapper)
        1    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/problem_definition.py:184(maxz)
        4    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/simplex_networks.py:154(find_pivot_row)
  

In [31]:
print(p_max_stats.sort_stats("cumulative").print_stats("simplex|lambda|maxz"))

Mon May 10 01:38:18 2021    maxz_stats

         212 function calls (192 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 41 to 6 due to restriction <'simplex|lambda|maxz'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/problem_definition.py:184(maxz)
        4    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/simplex_networks.py:154(find_pivot_row)
        2    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/simplex_networks.py:183(pivot)
        4    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/simplex_networks.py:98(find_negative_row)
        7    0.000    0.000    0.000    0.000 /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/simplex_networks.py:51(pivots_row)
      

In [32]:
print(p_max_stats.strip_dirs().sort_stats("cumulative").print_stats("lambda|simplex|maxz"))

Mon May 10 01:38:18 2021    maxz_stats

         212 function calls (192 primitive calls) in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 41 to 6 due to restriction <'lambda|simplex|maxz'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 problem_definition.py:184(maxz)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:154(find_pivot_row)
        2    0.000    0.000    0.000    0.000 simplex_networks.py:183(pivot)
        4    0.000    0.000    0.000    0.000 simplex_networks.py:98(find_negative_row)
        7    0.000    0.000    0.000    0.000 simplex_networks.py:51(pivots_row)
        1    0.000    0.000    0.000    0.000 simplex_networks.py:29(pivots_col)




In [33]:
# El número de veces que se llamaron a funciones primitivas o built in: 
print(p_max_stats.prim_calls)

192


In [34]:
# qué funciones llaman a otras
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                            was called by...
                                                                                        ncalls  tottime  cumtime
interactiveshell.py:3400(run_code)                                                  <- 
{built-in method builtins.exec}                                                     <-       2    0.000    0.001  interactiveshell.py:3400(run_code)
<ipython-input-26-4199b605bc38>:3(<module>)                                         <-       1    0.000    0.001  {built-in method builtins.exec}
line_profiler.py:124(wrapper)                                                       <-       1    0.000    0.001  <ipython-input-26-4199b605bc38>:3(<module>)
                                                                                            12    0.000    0.000  problem_definition.py:184(maxz)
                                                                   

In [35]:
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                            was called by...
                                                                                        ncalls  tottime  cumtime
interactiveshell.py:3400(run_code)                                                  <- 
{built-in method builtins.exec}                                                     <-       2    0.000    0.001  interactiveshell.py:3400(run_code)
<ipython-input-26-4199b605bc38>:3(<module>)                                         <-       1    0.000    0.001  {built-in method builtins.exec}
line_profiler.py:124(wrapper)                                                       <-       1    0.000    0.001  <ipython-input-26-4199b605bc38>:3(<module>)
                                                                                            12    0.000    0.000  problem_definition.py:184(maxz)
                                                                   

In [36]:
# Análogamente podemos establecer los filtros usados en print_stats
p_max_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 41 to 10 due to restriction <10>

Function                                     was called by...
                                                 ncalls  tottime  cumtime
interactiveshell.py:3400(run_code)           <- 
{built-in method builtins.exec}              <-       2    0.000    0.001  interactiveshell.py:3400(run_code)
<ipython-input-26-4199b605bc38>:3(<module>)  <-       1    0.000    0.001  {built-in method builtins.exec}
line_profiler.py:124(wrapper)                <-       1    0.000    0.001  <ipython-input-26-4199b605bc38>:3(<module>)
                                                     12    0.000    0.000  problem_definition.py:184(maxz)
                                                      8    0.000    0.000  simplex_networks.py:154(find_pivot_row)
problem_definition.py:184(maxz)              <-       1    0.000    0.000  line_profiler.py:124(wrapper)
simplex_networks.py:154(find_pivot_row)      <-       4    0.000  

In [37]:
p_max_stats.strip_dirs().sort_stats("cumulative").print_callees("maxz|lambda")

   Ordered by: cumulative time
   List reduced from 41 to 1 due to restriction <'maxz|lambda'>

Function                         called...
                                     ncalls  tottime  cumtime
problem_definition.py:184(maxz)  ->       2    0.000    0.000  <__array_function__ internals>:2(where)
                                         12    0.000    0.000  line_profiler.py:124(wrapper)
                                          2    0.000    0.000  {built-in method builtins.len}
                                          2    0.000    0.000  {built-in method builtins.max}
                                          2    0.000    0.000  {built-in method builtins.sum}




# Con line profiler

In [38]:
!pip install line_profiler --quiet

In [39]:
import line_profiler

In [40]:
line_prof = line_profiler.LineProfiler()

print(line_prof(maxz)(matrix_max_approx_1))

{'x1': 2.0, 'x2': 6.0, 'max': 36.0}


In [41]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 4e-06 s
File: /home/ubuntu/.local/lib/python3.8/site-packages/line_profiler/line_profiler.py
Function: wrapper at line 124

Line #      Hits         Time  Per Hit   % Time  Line Contents
   124                                                   @functools.wraps(func)
   125                                                   def wrapper(*args, **kwds):
   126                                                       self.enable_by_count()
   127         1          2.0      2.0     50.0              try:
   128         1          2.0      2.0     50.0                  result = func(*args, **kwds)
   129                                                       finally:
   130                                                           self.disable_by_count()
   131                                                       return result

None


## Podemos agregar todas las funciones en el lineprofiler

In [41]:
import line_profiler
from mex.simplex.maximizer_class import Maximizer

In [26]:
n_var_approx = 20
n_cons_approx = 30

In [47]:
# Todo junto
line_prof_all = line_profiler.LineProfiler()

line_prof_all.add_function(Maximizer.__init__)
#maxim = Maximi
line_prof_all(Maximizer.__init__)(Maximizer, 2, 3)
line_prof_all.add_function(Maximizer.add_constraint)
line_prof_all(Maximizer.add_constraint)(Maximizer, '1,0,L,4')
line_prof_all(Maximizer.add_constraint)(Maximizer, '0,2,L,12')
line_prof_all(Maximizer.add_constraint)(Maximizer, '3,2,L,18')
line_prof_all.add_function(Maximizer.add_objective)
line_prof_all(Maximizer.add_objective)(Maximizer, '3,5,0')
line_prof_all.add_function(Maximizer.solve)
line_prof_all(Maximizer.solve)(Maximizer)

print(line_prof_all.print_stats())

Timer unit: 1e-06 s

Total time: 1.7e-05 s

Could not find file /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/maximizer_class.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           
     9         1          2.0      2.0     11.8  
    10         1          1.0      1.0      5.9  
    11         1         12.0     12.0     70.6  
    12         1          1.0      1.0      5.9  
    13         1          1.0      1.0      5.9  

Total time: 0.000229 s

Could not find file /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/maximizer_class.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15               

In [105]:
# Todo junto
line_prof_all = line_profiler.LineProfiler()

line_prof_all.add_function(create_matrix)
matrix_max_approx_1 = line_prof_all(create_matrix)(n_var_approx_1,n_cons_approx_1)
line_prof_all.add_function(constrain)
line_prof_all(constrain)(matrix_max_approx_1,'1,0,L,4')
line_prof_all(constrain)(matrix_max_approx_1,'0,2,L,12')
line_prof_all(constrain)(matrix_max_approx_1,'3,2,L,18')
line_prof_all.add_function(obj)
line_prof_all(obj)(matrix_max_approx_1,'3,5,0')
line_prof_all.add_function(maxz)          
problem_approx_1 = line_prof_all(maxz)(matrix_max_approx_1)

In [106]:
print(line_prof_all.print_stats())

Timer unit: 1e-06 s

Total time: 0.000275 s

Could not find file /home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/problem_definition.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                           
    39                                           
    40                                           
    41                                           
    42                                           
    43                                           
    44                                           
    45                                           
    46                                           
    47                                           
    48                                           
    49                                           
    50                                         

## Utilizando kernprof

In [56]:
import os
os.chdir("..")
current_path = os.getcwd()

In [55]:
## Tuve que hacer sudo apt install python3-line-profiler -y en mi linux (hay que hacerlo en la imagen)

In [96]:
%%file ./src/mex/simplex/problem_definition_profile.py

import numpy as np
import logging

from mex.utils.general_profile import gen_var, convert_min, convert
from mex.simplex.simplex_networks_profile import pivots_col, find_pivot_col, pivots_row, pivot, find_pivot_row

@profile
def add_cons(matrix):
    """
    Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
    
    Returns:
    
        Flag (bool): True or False indicating whether 1+ constraints can be added.
    """

    lr = len(matrix[:, 0])
    empty = []

    for i in range(lr):
        total = 0
        for j in matrix[i, :]:
            total += j**2
        if total == 0:
            empty.append(total)

    if len(empty) > 1:
        return True
    else:
        return False

@profile
def constrain(matrix, eq, memory_prof=False):
    """
    Adds constraints to the problem.
    
    Args:
        matrix (numpy array): matrix defined with :mod:`create_matrix`.
        eq (string): coefficients of constraints expressions. Use **L** for *less than*, **G** for *greater than*,
        **E** for *equal to*.
    
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'3,2,G,18')  # 3x_1 + 2x_2 >= 18
    """

    if 'E' in eq:
        if add_cons(matrix):
            lc = len(matrix[0, :])
            lr = len(matrix[:, 0])
            var = lc - lr - 1
            j = 0

            while j < lr:
                row_check = matrix[j,:]
                total = 0
                for i in row_check:
                    total += float(i**2)
                if total == 0:
                    row = row_check
                    break
                j += 1
                
            eq = convert(eq)
            i = 0
            
            while i<len(eq)-1:
                row[i] = eq[i]
                i += 1
            
            row[-1] = eq[-1]
            #row[var+j] = 1
        
        else:
            logging.info('Cannot add another constraint.')
        
    else:
        if add_cons(matrix):
            lc = len(matrix[0, :])
            lr = len(matrix[:, 0])
            var = lc - lr -1
            j = 0
            
            while j < lr:
                row_check = matrix[j, :]
                total = 0
                for i in row_check:
                    total += float(i**2)
                if total == 0:
                    row = row_check
                    break
                j +=1
                
            eq = convert(eq)
            i = 0
            
            while i<len(eq)-1:
                row[i] = eq[i]
                i += 1
            
            row[-1] = eq[-1]
            row[var+j] = 1
            
        else:
            logging.info('Cannot add another constraint.')
    if memory_prof:
        return matrix

    if memory_prof:
        return matrix

@profile
def add_obj(matrix):
    """
    Verifies if the objective function can be added.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
    
    Returns:
    
        Flag (bool): True or False indicating whether objective function can be added.
    """
    
    lr = len(matrix[:,0])
    empty = []
    
    for i in range(lr):
        total = 0
        for j in matrix[i, :]:
            total += j**2
        if total == 0:
            empty.append(total)
    
    if len(empty) == 1:
        return True
    else:
        return False

@profile
def obj(matrix,eq,memory_prof=False):
    """
    Adds the objective function to the problem matrix.
    
    .. note::
        Objective function must be added **after** constraints have been input.
    
    Args:
    
        matrix (numpy array): matrix defined with :mod:`create_matrix`.
        
        eq (string): coefficients of objective function.
    
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'4,2,G,18')  # 4x_1 + 2x_2 >= 18
    >>> obj(problem_matrix,'3,5,0')           # 3x_1 + 5x_2
    """
    
    if add_obj(matrix):
        eq = [float(i) for i in eq.split(',')]
        lr = len(matrix[:,0])
        row = matrix[lr-1,:]
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        logging.info('You must finish adding constraints before the objective function can be added.')

    if memory_prof:
        return matrix

@profile
def maxz(matrix, aux=True):
    """
    Creates maximization function. Determines if 1 extra pivot is required, locates the pivot element,
    pivots about it and continues the process until all negative elements have been removed from
    the last column and row.
    
    Args:
    
        matrix (numpy array): problem matrix with constraints and objective function added.
    
    Returns:
    
        *(dict)* A dictionary with Max and variables.
        
    >>> problem_matrix = create_matrix(2,3)   # 2 variables and 3 constraints
    >>> constrain(problem_matrix,'1,L,4')     # x_1 <= 4
    >>> constrain(problem_matrix,'0,2,L,12')  # 2x_2 <= 12
    >>> constrain(problem_matrix,'4,2,G,18')  # 4x_1 + 2x_2 >= 18
    >>> obj(problem_matrix,'3,5,0')           # 3x_1 + 5x_2
    >>> maxz(problem_matrix)
    {'x1': 4.0, 'x2': 6.0, 'max': 42.0}
    """
    
    while pivots_col(matrix):
        matrix = pivot(find_pivot_col(matrix)[0], find_pivot_col(matrix)[1], matrix)
    while pivots_row(matrix):
        matrix = pivot(find_pivot_row(matrix)[0], find_pivot_row(matrix)[1], matrix)
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    
    for i in range(var):
        col = matrix[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(matrix)[i]] = matrix[loc, -1]
        else:
            val[gen_var(matrix)[i]] = 0
    val['max'] = matrix[-1, -1]
    
    return val

@profile
def minz(matrix, aux=True):
    """
    Creates minimization function. Determines if 1 extra pivot is required, locates the pivot element,
    pivots about it and continues the process until all negative elements have been removed from 
    the last column and row.
    
    Args:
    
        matrix (numpy array): problem matrix with constraints and objective function added.
    
    Returns:
    
        (dict) A dictionary with Min and variables.
        
    >>> problem_matrix = create_matrix(2,4)   # 2 variables and 4 constraints
    >>> constrain(problem_matrix,'1,1,L,6')   # x_1 + x_2 <= 6
    >>> constrain(problem_matrix,'-1,2,L,8')  # -x_1 + 2x_2 <= 8
    >>> constrain(problem_matrix,'1,G,0')     # x_1 >= 0
    >>> constrain(problem_matrix,'0,1,G,0')   # x_2 >= 0
    >>> obj(problem_matrix,'-1,-3,0')         # -x_1 - 3x_2
    >>> minz(problem_matrix)
    {'x1': 1.3333333333333333, 'x2': 4.666666666666667}
    """

    matrix = convert_min(matrix)
    while pivots_col(matrix):
        matrix = pivot(find_pivot_col(matrix)[0], find_pivot_col(matrix)[1], matrix)
    while pivots_row(matrix):
        matrix = pivot(find_pivot_row(matrix)[0], find_pivot_row(matrix)[1], matrix)
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    
    for i in range(var):
        col = matrix[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(matrix)[i]] = matrix[loc, -1]
        else:
            val[gen_var(matrix)[i]] = 0
    val['min'] = matrix[-1, -1]*-1

    if aux:
        return val


Overwriting ./src/mex/simplex/problem_definition_profile.py


In [97]:
%%file src/mex/simplex/simplex_networks_profile.py
import numpy as np
import logging

@profile
def create_matrix(variables, constraints):
    """
    Creates a matrix with enough rows for each constraint plus the objective function
    and enough columns for all the variables.
    
    Args:
    
        variables (int): number of variables.
        
        constraints (int): number of constraints.
    
    Returns:
    
        matrix (numpy array): zero matrix.
    """
    
    matrix = np.zeros((constraints + 1, variables + constraints + 2))
    
    return matrix

@profile
def pivots_col(matrix):
    """
    Checks to see if pivots are required due to negative values in right column,
    excluding the bottom value.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        Flag (bool): True or False indicating whether a negative element was found.
    """
    
    m = min(matrix[:-1, -1])
    if m >= 0:
        return False
    else:
        return True

@profile
def pivots_row(matrix):
    """
    Checks to see if pivots are required due to negative values in bottom row, 
    excluding the final value.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        Flag (bool): True or False indicating whether a negative element was found.
    """
    
    l = len(matrix[:, 0])
    m = min(matrix[l-1, :-1])
    if m >= 0:
        return False
    else:
        return True

@profile
def find_negative_col(matrix):
    """
    Finds location of negative values in right column.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        n (int): index of negative value in right column.
    """
    
    l = len(matrix[0, :])
    m = min(matrix[:-1, l-1])
    if m <= 0:
        n = np.where(matrix[:-1, l-1] == m)[0][0]
    else:
        n = None
    
    return n

@profile
def find_negative_row(matrix):
    """
    Finds location of negative values in bottom row.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        n (int): index of negative value in bottom row.
    """
    
    l = len(matrix[:,0])
    m = min(matrix[l-1,:-1])
    if m <= 0:
        n = np.where(matrix[l-1, :-1] == m)[0][0]
    else:
        n = None
    
    return n

@profile
def find_pivot_col(matrix):
    """
    Finds pivot element corresponding to a negative value in right column.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
        
        index (int): index of most negative value.
        
        c (int): index smallest value on row.
    """
    
    total = []
    neg = find_negative_col(matrix)
    row = matrix[neg, :-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = matrix[:-1, c]
    for i,j in zip(col, matrix[:-1,-1]):  #i for col with neg, j for right col
        if i != 0 and j/i > 0:
            total.append(j/i)
        else:
            total.append(10000) #placeholder, might need to update for large scale
    index = total.index(min(total))
    
    return [index,c]

@profile
def find_pivot_row(matrix):
    """
    Finds pivot element corresponding to a negative value in bottom row.
    
    Args:
        
        matrix (numpy array): matrix to be reviewed.
        
    Returns:
    
        index (int): index of most negative value.
        
        neg (int): index smallest value.
    """
    
    if pivots_row(matrix):
        total = []
        neg = find_negative_row(matrix)
        for i, j in zip(matrix[:-1, neg], matrix[:-1, -1]):
            if i != 0 and j/i > 0:
                total.append(j/i)
            else:
                total.append(10000) #placeholder, might need to update for large scale
        index = total.index(min(total))
        
        return [index,neg]

@profile
def pivot(row,col,matrix):
    """
    Pivot about a value to remove negative in final column or row.
    
    Args:
    
        matrix (numpy array): matrix to be reviewed.
        
        row (int): position to pivot.
        
        col (int): position to pivot.
        
    Returns:
    
        t (numpy array): updated matrix.
    """
    
    lr = len(matrix[:, 0])
    lc = len(matrix[0, :])
    t = np.zeros((lr, lc))
    pr = matrix[row, :]
    if matrix[row, col]**2 > 0:
        e = 1/matrix[row, col]
        r = pr*e
        for i in range(len(matrix[:, col])):
            k = matrix[i, :]
            c = matrix[i, col]
            if list(k) == list(pr):
                continue
            else:
                t[i, :] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        logging.info('Cannot pivot on this element')


Overwriting src/mex/simplex/simplex_networks_profile.py


In [98]:
%%file src/mex/utils/general_profile.py

@profile
def convert_min(matrix):
    """
    This function multiplies by -1 the objective function for maximization problems. This is because
    if the problem to be solved is maximization then it is analogue to solve the problem -minimization.
    
    Args:
    
        matrix (numpy array): matrix to be updated.
    
    Returns:
    
        matrix (numpy array): matrix multiplied by -1.
    """
    
    matrix[-1, :-2] = [-1*i for i in matrix[-1, :-2]]
    matrix[-1, -1] = -1*matrix[-1, -1]
    
    return matrix

@profile
def gen_var(matrix):
    """
    Generates the required number of variables. They are defined by the problem.
    
    Args:
    
        matrix (numpy array): matrix to be updated.
    
    Returns:
    
        v (list): list with problem variables.
    """
    
    lc = len(matrix[0, :])
    lr = len(matrix[:, 0])
    
    var = lc - lr -1
    v = []
    
    for i in range(var):
        v.append('x' + str(i+1))
    
    return v

@profile
def convert(eq):
    """
    Converts equation into a list containing the coefficients of the equation.
    
    Args:
    
        eq (string): equation defined with :func:`constrain`.
    
    Returns:
    
        eq (list): list with equation coefficients.
    """
    
    eq = eq.split(',')
    
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i)*-1 for i in eq]
        return eq
    
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq
    
    if 'E' in eq:
        l = eq.index('E')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq


Overwriting src/mex/utils/general_profile.py


In [99]:
%%file src/mex/simplex/maximizer_class_profile.py

import numpy as np
from mex.simplex.simplex_networks_profile import create_matrix
from mex.simplex.problem_definition_profile import constrain, obj, maxz

@profile
class Maximizer:
    max = None
    coeff = None
    """Maximize the objective function"""
    def __init__(self, n_variables, n_constraints):
        self.n_variables = n_variables
        self.n_constraints = n_constraints
        self.matrix = create_matrix(self.n_variables, self.n_constraints)
        self.max = None
        self.coeff = None

    def add_constraint(self, constraint):
        constrain(self.matrix, constraint)

    def add_objective(self, objective):
        obj(self.matrix, objective)

    def solve(self):
        solve = maxz(self.matrix)
        self.max = solve['max']
        self.coeff = np.array(list(solve.values()))[:-1]

    def get_max(self):
        return self.max

    def get_coeff(self):
        return self.coeff


Overwriting src/mex/simplex/maximizer_class_profile.py


### Para maxz

In [111]:
!export PYTHONPATH=$PWD

In [77]:
path = os.path.join(current_path,'src','profiling','tiempo','max_class_profiling.py')

In [113]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to max_class_profiling.py.lprof
Timer unit: 1e-06 s

Total time: 0 s
File: /home/ubuntu/practica-2-segunda-parte-caroacostatovany/src/profiling/tiempo/general_profile.py
Function: convert_min at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profile
     3                                           def convert_min(matrix):
     4                                               """
     5                                               This function multiplies by -1 the objective function for maximization problems. This is because
     6                                               if the problem to be solved is maximization then it is analogue to solve the problem -minimization.
     7                                               
     8                                               Args:
     9                                               
    10                                               

### Para minz

In [8]:
path = os.path.join(current_path,'src','profiling','tiempo','min_line_profiler.py')

In [111]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to min_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 7.7e-05 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: add_cons at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def add_cons(matrix):
    10                                               """
    11                                               Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    12                                               0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    13                                               
    14                                               Args:
    15                                               
    16     

### Para la red con problema de minimización

In [113]:
path = os.path.join(current_path,'src','profiling','tiempo','net_line_profiler.py')

In [114]:
%%bash -s $path
kernprof -l -v "$1"

Wrote profile results to net_line_profiler.py.lprof
Timer unit: 1e-06 s

Total time: 0.001676 s
File: /home/user/.pyenv/versions/3.7.4/envs/optimizacion/lib/python3.7/site-packages/mex/simplex/problem_definition.py
Function: add_cons at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def add_cons(matrix):
    10                                               """
    11                                               Checks if 1 extra constraint can be added to the matrix, this means that there are at least two rows of all
    12                                               0 elements. If this condition is not satisfied, our program will not allow the user to add additional constraints.
    13                                               
    14                                               Args:
    15                                               
    16    

/home/user/Documents/Caro_MCD/Optimizacion/practica-2-segunda-parte-caroacostatovany/src/profiling/tiempo/net_line_profiler.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  net_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).fun
/home/user/Documents/Caro_MCD/Optimizacion/practica-2-segunda-parte-caroacostatovany/src/profiling/tiempo/net_line_profiler.py:23: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  net_coeff_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).x
